## Setup

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 4.2MB/s 
     |████████████████████████████████| 1.2MB 33.5MB/s 
     |████████████████████████████████| 204kB 34.7MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
Mounted at /content/gdrive


In [ ]:
from fastbook import *

## Importing data and defning DataLoaders object

In [ ]:
path = untar_data(URLs.MNIST)

In [ ]:
path.ls()

(#2) [Path('/root/.fastai/data/mnist_png/testing'),Path('/root/.fastai/data/mnist_png/training')]

In [ ]:
images = DataBlock(
    blocks = (ImageBlock(cls=PILImageBW), CategoryBlock),
    get_items = get_image_files,
    splitter = GrandparentSplitter(train_name='training', valid_name='testing'),
    get_y = parent_label
)

In [ ]:
dls = images.dataloaders(path)

## Defining the Neural Network

In [ ]:
neural_network = nn.Sequential(
    Flatten(), ## because we were unable to use view on the DataBlock to create a single rank tensor for each image
    nn.Linear(28*28, 30),
    nn.ReLU(),
    nn.Linear(30,10)
)

## Defining the Loss Function

In [ ]:
nn.CrossEntropyLoss()

CrossEntropyLoss()

## Defining the Optimizer

In [ ]:
class Optim:
    def __init__(self, params, lr):
        self.params = list(params)
        self.lr = lr
    
    def step(self):
        for p in self.params:
            p.data -= self.lr * p.grad

    def zero_grad(self):
        for p in self.params:
            p.grad = None

## Defining Learner

In [ ]:
class Learner:
    def __init__(self, dataloaders, model, opt_class, loss_function, learning_rate=0.1):
        self.train_dl = dataloaders[0]
        self.valid_dl = dataloaders[1]
        self.model = model
        self.opt_function = opt_class(self.model.parameters(), learning_rate)
        self.loss_function = loss_function

    def train_epoch(self):
        for xb, yb in self.train_dl:
            predictions = self.model(xb)
            loss = self.loss_function(predictions, yb)
            loss.backward()
            self.opt_function.step()
            self.opt_function.zero_grad()
    
    def batch_accuracy(self, predictions, yb):
        probabilities = predictions.softmax(1)
        predicted = probabilities.argmax(1)
        correct = predicted == yb
        return correct.float().mean()
    
    def validate_epoch(self):
        accuracy = [self.batch_accuracy(self.model(xb), yb) for xb, yb in self.valid_dl]
        return round(torch.stack(accuracy).mean().item(),4)

    def fit(self, n):
        for i in range(n):
            self.train_epoch()
            print(self.validate_epoch())

## Putting it all together

In [ ]:
lr = 0.1
learn = Learner(dls, neural_network.to('cuda'), Optim, nn.CrossEntropyLoss(), lr)

In [ ]:
learn.fit(10)

0.9198
0.9352
0.9447
0.9505
0.952
0.9568
0.9592
0.961
0.962
0.9635
